In [92]:
import pandas as pd
import numpy as np
import dash
from dash import Dash, html, dcc, Input, Output, ctx, callback
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

In [93]:
date_df = pd.read_csv('https://raw.githubusercontent.com/BabbarGaurav/Dash/main/Input%20Files/dim_date.csv?token=GHSAT0AAAAAACI4MUWAWPOFUH3H3SE7IHRUZJMHZNQ')
hotel_df = pd.read_csv('https://raw.githubusercontent.com/BabbarGaurav/Dash/main/Input%20Files/dim_hotels.csv?token=GHSAT0AAAAAACI4MUWBZNIZZ7ZAZVSWUAH4ZJMHZTQ')
room_df = pd.read_csv('https://raw.githubusercontent.com/BabbarGaurav/Dash/main/Input%20Files/dim_rooms.csv?token=GHSAT0AAAAAACI4MUWATADK4QNXUY64FLXGZJMH2WA')
agg_df = pd.read_csv('https://raw.githubusercontent.com/BabbarGaurav/Dash/main/Input%20Files/fact_aggregated_bookings.csv?token=GHSAT0AAAAAACI4MUWAQUQ7OP7ZKL3B2HLIZJMH26Q')
booking_df = pd.read_csv('https://raw.githubusercontent.com/BabbarGaurav/Dash/main/Input%20Files/fact_bookings.csv?token=GHSAT0AAAAAACI4MUWAUR2KXTZ3LGFM3LFOZJMH3JA')

In [94]:
# Convert the 'date' column to datetime data type
date_df['date'] = pd.to_datetime(date_df['date'])

# Extract and convert 'week no' to an integer
date_df['week no'] = date_df['week no'].str.extract('(\d+)').astype(int)

#Including a new column to identify which are sundays
date_df['day_of_week'] = date_df['date'].dt.day_name()

#Correcting day_type based on information provided by stackholders about sunday
date_df['day_type'] = date_df['day_type'].replace({'weekeday': 'weekday'})
date_df.loc[date_df['day_of_week'] == 'Sunday', 'day_type'] = 'weekday'

# Convert 'check_in_date' to datetime data type
agg_df['check_in_date'] = pd.to_datetime(agg_df['check_in_date'])

# Convert date columns to datetime data type
booking_df['booking_date'] = pd.to_datetime(booking_df['booking_date'])
booking_df['check_in_date'] = pd.to_datetime(booking_df['check_in_date'])
booking_df['checkout_date'] = pd.to_datetime(booking_df['checkout_date'])


date_df = date_df.rename(columns={'date': 'check_in_date'})
room_df = room_df.rename(columns={'room_id': 'room_category'})

#merge df
booking_df = pd.merge(booking_df, date_df, on='check_in_date', how='inner')

booking_df = pd.merge(booking_df, hotel_df, on='property_id', how='inner')

booking_df = pd.merge(booking_df, room_df, on='room_category', how='inner')

booking_df = pd.merge(booking_df, agg_df, on = ['property_id','check_in_date', 'room_category'], how = 'inner')

booking_df["month"] = booking_df["check_in_date"].dt.strftime("%B")
booking_df = booking_df.drop(columns=["mmm yy"])

#Measures

In [105]:
total_revenue = booking_df['revenue_realized'].sum()

total_bookings = len(booking_df)

total_capacity = agg_df['capacity'].sum()

total_successful_bookings = booking_df.groupby(['property_id', 'check_in_date','room_category']).size().reset_index(name='successful_booking')['successful_booking'].sum()

occupancy_percentage = (total_successful_bookings / total_capacity) * 100

average_rating = booking_df['ratings_given'].mean()

no_of_days = booking_df['check_in_date'].max() - booking_df['check_in_date'].min()

total_cancelled_bookings = len(booking_df[booking_df['booking_status'] == 'Cancelled'])

cancellation_percentage = (total_cancelled_bookings / total_bookings) * 100

total_checked_out = len(booking_df[booking_df['booking_status'] == 'Checked out'])

total_no_show_bookings = len(booking_df[booking_df['booking_status'] == 'No Show'])

no_show_rate_percentage = (total_no_show_bookings / total_bookings) * 100

booking_platform_percentages = booking_df['booking_platform'].value_counts(normalize = True) * 100

room_class_percentages = booking_df['room_class'].value_counts(normalize = True)*100

adr = booking_df['revenue_generated'].sum() / total_bookings

realisation_percentage = total_checked_out / total_bookings

revpar = total_revenue / total_capacity

In [249]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

logo = 'https://d1muf25xaso8hp.cloudfront.net/https%3A%2F%2Ff2fa1cdd9340fae53fcb49f577292458.cdn.bubble.io%2Ff1663995322968x622558936746862000%2FAtliq%2520A%2520logo-04.png'


def your_filtering_function(selected_cities, selected_room_classes, selected_months):
    # Filter the data based on selected cities and room classes
    if not selected_cities and not selected_room_classes and not selected_months:
        # No filters selected, return total revenue for all data
        return booking_df
    else:
        # Apply filters based on selected cities and room classes
        filtered_data = booking_df
        if selected_cities:
            filtered_data = filtered_data[filtered_data['city'].isin(selected_cities)]
        if selected_room_classes:
            filtered_data = filtered_data[filtered_data['room_class'].isin(selected_room_classes)]
        if selected_months:
            filtered_data = filtered_data[filtered_data['month'].isin(selected_months)]

        return filtered_data

# Define the content of the navbar
navbar = dbc.Navbar(
    id='navbar',
    children=[
        dbc.Row([
            dbc.Col(
                html.Img(
                    src=logo,
                    height="50px",
                ),
            ),
            dbc.Col(
                dbc.NavbarBrand("Analytics Dashboard", style={"color": "Black", "fontSize": "30px", "fontFamily": "Helvetica", "fontWeight": "bold"}),
            ),
        ],
    )], color = '#f5d3cb')

# Create a card component without the extra level of nesting
chipgroup = dbc.CardBody(
    [
        dbc.Row([
            dbc.Col([
                html.H6('Filter by City'),
                dmc.ChipGroup(
                    [
                        dmc.Chip(
                            x,
                            value = x,
                            variant = 'filled',
                            size = 'sm',
                            radius = 'sm',
                        )
                        for x in hotel_df.sort_values('city')['city'].unique()
                    ],
                    id = 'city-filter',
                    multiple=True,
                )
            ])
            ]),
        
        html.Br(),

        dbc.Row([
            dbc.Col([
                html.H6('Filter by Room Category'),
                dmc.ChipGroup(
                    [
                        dmc.Chip(
                            x,
                            value = x,
                            variant = 'filled',
                            size = 'sm',
                            radius = 'sm',
                        )
                        for x in room_df.sort_values('room_class')['room_class'].unique()
                    ],
                    id='room-class-filter',
                    multiple=True,
                )
            ])
            ]),
        html.Br(),

        dbc.Row([
            dbc.Col([
                html.H6('Filter by Month'),
                dmc.ChipGroup(
                    [
                        dmc.Chip(
                            x,
                            value = x,
                            variant = 'filled',
                            size = 'sm',
                            radius = 'sm',
                        )
                        for x in booking_df['month'].unique()
                    ],
                    id='month-filter',
                    multiple=True,
                )
        ])
    ])
])

filters = dmc.Navbar(
            p="md",
            width={"base": 300},
            height=500,
            fixed=False,
            position={"right": 0, "top": 400},
            children=[
               dbc.Card(chipgroup)
            ]
)

layout = html.Div([
        html.Br(),
                dbc.Card(id='card_num1', style = {'height':'150px'}),
                dbc.Card(id='card_num2'),
                dbc.Card(id='card_num3'),
                dbc.Card(id='card_num4')]
            , style = {'display':'flex'},
)




# Define the layout of your dashboard
app.layout = html.Div(children=[
    navbar,
    html.Div(
        children=[filters, layout],
        style={"display": "flex"},
        
    )
])



# Create a callback function to update the filter values
@app.callback([Output('card_num1', 'children'),
               Output('card_num2', 'children'),
               Output('card_num3', 'children'),
               Output('card_num4', 'children')],
              [Input('city-filter', 'value'),
               Input('room-class-filter', 'value'),
               Input('month-filter', 'value')])
               

def update_filtered_data(selected_cities, selected_room_classes,selected_months):

    # Get the filtered data using the filtering function
    filtered_data = your_filtering_function(selected_cities, selected_room_classes, selected_months)

    # Calculate the total revenue based on the filtered data
    total_revenue_filtered = filtered_data['revenue_realized'].sum()
    occupancy_percentage_ = (filtered_data.groupby(['property_id', 'check_in_date','room_category']).size().reset_index(name='successful_booking')['successful_booking'].sum() / total_capacity) * 100
    average_rating_ = filtered_data['ratings_given'].mean()
    cancellation_percentage_ = (len(filtered_data[filtered_data['booking_status'] == 'Cancelled']) / len(filtered_data)) * 100
    
    
    
    fig1 = go.Figure(go.Indicator(
                            mode = "number",
                            value = total_revenue_filtered,
                            number = {'prefix': "INR "}
                                )
                   )
    fig1.update_layout(
        height=100,  # You can adjust the height as needed
        margin=dict(l = 20, r = 20, t = 20, b = 50),
    )    
    
    fig2 = go.Figure(go.Indicator(
                            mode = "number",
                            value = occupancy_percentage_,
                            number = {'suffix': "%"}
                                )
                   )
    fig2.update_layout(
        height=100,  # You can adjust the height as needed
        margin=dict(l = 20, r = 20, t = 20, b = 50),
    )  
    
    fig3 = go.Figure(go.Indicator(
                            mode = "number",
                            value = average_rating_,
                                )
                   )
    fig3.update_layout(
        height=100,  # You can adjust the height as needed
        margin=dict(l = 20, r = 20, t = 20, b = 50),
    )
    
    fig4 = go.Figure(go.Indicator(
                            mode = "number",
                            value = cancellation_percentage_,
                            number = {'suffix': "%"}
                                )
                   )
    fig4.update_layout(
        height=100,  # You can adjust the height as needed
        margin=dict(l = 20, r = 20, t = 20, b = 50),
    )  


    # Define the content for card1 & card2
    
    card1_content = [
        dbc.CardBody(
                [
                html.H6('Total Revenue', style = {'fontWeight':'bold', 'fontSize':'18px','textAlign':'center'}),
                dcc.Graph(figure = fig1)
                ]),  
    ]
    
    card2_content = [
        dbc.CardBody(
                [
                html.H6('Occupancy %', style = {'fontWeight':'bold', 'fontSize':'18px','textAlign':'center'}),
                dcc.Graph(figure = fig2)
                ]),  
    ]
    
    card3_content = [
        dbc.CardBody(
                [
                html.H6('Average Rating', style = {'fontWeight':'bold', 'fontSize':'18px','textAlign':'center'}),
                dcc.Graph(figure = fig3)
                ]),  
    ]
        
    card4_content = [
        dbc.CardBody(
                [
                html.H6('Cancellation Rate', style = {'fontWeight':'bold', 'fontSize':'18px','textAlign':'center'}),
                dcc.Graph(figure = fig4)
                ]),  
    ]
    
    return card1_content, card2_content, card3_content, card4_content


# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

logo = 'https://d1muf25xaso8hp.cloudfront.net/https%3A%2F%2Ff2fa1cdd9340fae53fcb49f577292458.cdn.bubble.io%2Ff1663995322968x622558936746862000%2FAtliq%2520A%2520logo-04.png'


def your_filtering_function(selected_cities, selected_room_classes):
    # Filter the data based on selected cities and room classes
    if not selected_cities and not selected_room_classes:
        # No filters selected, return total revenue for all data
        return booking_df
    else:
        # Apply filters based on selected cities and room classes
        filtered_data = booking_df
        if selected_cities:
            filtered_data = filtered_data[filtered_data['city'].isin(selected_cities)]
        if selected_room_classes:
            filtered_data = filtered_data[filtered_data['room_class'].isin(selected_room_classes)]

        return filtered_data

# Define the content of the navbar
navbar = dbc.Navbar(
    dbc.Row(
        [
            dbc.Col(
                html.Img(
                    src=logo,
                    height="50px",
                ),
            ),
            dbc.Col(
                dbc.NavbarBrand("Analytics Dashboard", style={"color": "Black", "fontSize": "30px", "fontFamily": "Helvetica", "fontWeight": "bold"}),
                width = {"size": 1},
            ),
        ],
        align="center",
    ), color = '#f5d3cb'
)

# Create a card component without the extra level of nesting
filters = dbc.CardBody(
    [
        dbc.Row([
            dbc.Col([
                html.H6('Filter by City'),
                dcc.Dropdown(
                    id='city-filter',
                    options=[
                        {'label': i, 'value': i} for i in hotel_df.sort_values('city')['city'].unique()
                    ],
                    multi=True,
                )
            ]),
            dbc.Col([
                html.H6('Filter by Room Category'),
                dcc.Dropdown(
                    id='room-class-filter',
                    options=[
                        {'label': i, 'value': i} for i in room_df.sort_values('room_class')['room_class'].unique()
                    ],
                    multi=True,
                
                    
                )
            ])
        ])
    ]) 


body_app = dbc.Container([
    
    html.Br(),
    
    dbc.Row([
        dbc.Col([dbc.Card(filters,style={'height':'150px'})]),
        dbc.Col([dbc.Card(id='card_num1', style={'height': '150px'})]),
            ],className="g-0"),
    
    html.Br(),

    ], 
    style = {'backgroundColor':'#f7f7f7'},
    fluid = True)


# Define the layout of your dashboard
app.layout = dbc.Container(id='parent', children=[navbar, body_app])



# Create a callback function to update the filter values
@app.callback([Output('card_num1', 'children')],
              [Input('city-filter', 'value'),
               Input('room-class-filter', 'value')])
               

def update_filtered_data(selected_cities, selected_room_classes):

    # Get the filtered data using the filtering function
    filtered_data = your_filtering_function(selected_cities, selected_room_classes)

    # Calculate the total revenue based on the filtered data
    total_revenue_filtered = filtered_data['revenue_realized'].sum()
    
    
    fig = go.Figure(go.Indicator(
                            mode = "number",
                            value = total_revenue_filtered,
                            number = {'prefix': "INR "}
                                )
                   )
    fig.update_layout(
        height=100,  # You can adjust the height as needed
        margin=dict(l = 20, r = 20, t = 20, b = 50),
    )    


    # Define the content for card1
    
    card1_content = [
        dbc.CardBody(
                [
                html.H6('Total Revenue', style = {'fontWeight':'bold', 'fontSize':'18px','textAlign':'center'}),
                dcc.Graph(figure = fig)
                ]),  
    ]
    
    return card1_content


# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)